In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarusr
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser
from scipy.stats import entropy

from sklearn.metrics.cluster import normalized_mutual_info_score
from itertools import combinations

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Wed Aug 10 16:50:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 29%   37C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 27%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.lda_topic_de
    
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'lda_topic_de')

# DE

In [6]:
de_topic = db_de.aggregate([
        {"$unwind" : "$classification"}
])


In [7]:
de_df = pd.DataFrame(de_topic)

In [8]:
# 取cpc前3碼
de_df["cpc_n"] = de_df.classification.apply(lambda x: x[0:3])

In [9]:
de_df

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/3234,None,6,G06
1,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/329,None,6,G06
2,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0216,None,6,H04
3,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04L43/08,None,6,H04
4,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0219,None,6,H04
...,...,...,...,...,...,...,...,...
8079,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/09,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8080,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/113,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8081,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/187,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8082,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H03H9/17,Es wird eine piezoelektrische Zusammensetzung ...,3,H03


In [66]:
de_df_2 = pd.DataFrame(list(db_de.find()))

In [67]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1
...,...,...,...,...,...,...,...
1060,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9
1061,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5
1062,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6
1063,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5


In [68]:
# de_df_2.classification = de_df_2.classification.fillna(0)

In [69]:
de_df_2 = de_df_2[de_df_2.classification.notna()]

In [70]:
de_df_2.reset_index(drop=True,inplace=True)

In [71]:
de_df_2.classification[0][0:3]

['G06F1/3234', 'G06F1/329', 'H04W52/0216']

In [72]:
# 取cpc前3碼
de_df_2["cpc_n"] = de_df_2.classification.apply(lambda x: x[0][0:3])

/tmp/ipykernel_9325/667598090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2["cpc_n"] = de_df_2.classification.apply(lambda x: x[0][0:3])


In [73]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6,G06
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10,C07
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4,G06
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11,G06
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1,G07
...,...,...,...,...,...,...,...,...
1045,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9,B60
1046,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5,A61
1047,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6,G06
1048,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5,C23


In [74]:
de_df2 = de_df_2.groupby(["cpc_n","topic"],as_index=False).size()

In [75]:
de_test = de_df2.groupby(["topic"])

In [76]:
de_df_3 = de_df2.sort_values('size',ascending = False)
group = de_df_3.groupby(['topic']).head(2)
group.sort_values('topic')

,cpc_n,topic,size
192,H03,0,1
93,F25,0,2
121,G06,1,40
204,H04,1,72
122,G06,2,33
205,H04,2,58
206,H04,3,12
123,G06,3,16
207,H04,4,18
124,G06,4,14


In [77]:
de_df2.sort_values('size',ascending = False).head(30)

,cpc_n,topic,size
204,H04,1,72
205,H04,2,58
213,H04,10,58
210,H04,7,50
121,G06,1,40
208,H04,5,39
211,H04,8,33
122,G06,2,33
130,G06,10,27
214,H04,11,26


## 建立Dictionary轉換DF

In [78]:
# CPC dict
cpc_dict = {
"A61":0,
"B60":1,
"C08":2,
"G01":3,
"G05":4,
"G06":5,
"G08":6,
"H01":7,
"H02":8,
"H03":9,
"H04":10,
"H05":11,
"Y02":12
}

In [79]:
# topic dict
de_dict={
    0:"G08",
    1:"G06",
    2:"G05",
    3:"B60",
    4:"C09",
    5:"G06",
    6:"Y02",
    7:"H04",
    8:"A47",
    9:"Y02",
    10:"H04",
    11:"H01",
    12:"H04"
}

In [80]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6,G06
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10,C07
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4,G06
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11,G06
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1,G07
...,...,...,...,...,...,...,...,...
1045,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9,B60
1046,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5,A61
1047,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6,G06
1048,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5,C23


In [81]:
de_dict.get(de_df_2.topic[0])

'Y02'

In [82]:
# topic to CPC
de_df_2["topic_to_cpc"] = de_df_2.topic.apply(lambda x: de_dict.get(x))

/tmp/ipykernel_9325/700430810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2["topic_to_cpc"] = de_df_2.topic.apply(lambda x: de_dict.get(x))


In [83]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6,G06,Y02
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10,C07,H04
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4,G06,C09
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11,G06,H01
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1,G07,G06
...,...,...,...,...,...,...,...,...,...
1045,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9,B60,Y02
1046,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5,A61,G06
1047,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6,G06,Y02
1048,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5,C23,G06


In [84]:
# cpc to num
de_df_2["cpc_to_num"] = de_df_2.cpc_n.apply(lambda x: cpc_dict.get(x))
de_df_2["topic_to_num"] = de_df_2.topic_to_cpc.apply(lambda x: cpc_dict.get(x))

/tmp/ipykernel_9325/2253347116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2["cpc_to_num"] = de_df_2.cpc_n.apply(lambda x: cpc_dict.get(x))
/tmp/ipykernel_9325/2253347116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2["topic_to_num"] = de_df_2.topic_to_cpc.apply(lambda x: cpc_dict.get(x))


In [85]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc,cpc_to_num,topic_to_num
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6,G06,Y02,5.0,12.0
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10,C07,H04,NaN,10.0
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4,G06,C09,5.0,NaN
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11,G06,H01,5.0,7.0
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1,G07,G06,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...
1045,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9,B60,Y02,1.0,12.0
1046,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5,A61,G06,0.0,5.0
1047,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6,G06,Y02,5.0,12.0
1048,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5,C23,G06,NaN,5.0


In [86]:
print(sum(de_df_2.cpc_to_num.isna()))
print(sum(de_df_2.topic_to_num.isna()))

131
133


In [87]:
# fill NA
de_df_2.cpc_to_num.fillna(99,inplace=True)
de_df_2.topic_to_num.fillna(99,inplace=True)

/tmp/ipykernel_9325/2419913829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2.cpc_to_num.fillna(99,inplace=True)
/tmp/ipykernel_9325/2419913829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df_2.topic_to_num.fillna(99,inplace=True)


In [88]:
de_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc,cpc_to_num,topic_to_num
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6,G06,Y02,5.0,12.0
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10,C07,H04,99.0,10.0
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4,G06,C09,5.0,99.0
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11,G06,H01,5.0,7.0
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1,G07,G06,99.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
1045,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9,B60,Y02,1.0,12.0
1046,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5,A61,G06,0.0,5.0
1047,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6,G06,Y02,5.0,12.0
1048,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5,C23,G06,99.0,5.0


# NMI計算
參考  
- [Entropy and Mutual Information](https://datascience.oneoffcoder.com/normalized-entropy-mi.html)
- [問題在Data 2: 從訊息理論來看資料的影響](https://chih-sheng-huang821.medium.com/%E7%AC%A8%E8%9B%8B-%E5%95%8F%E9%A1%8C%E5%9C%A8data-2-%E5%BE%9E%E8%A8%8A%E6%81%AF%E7%90%86%E8%AB%96%E4%BE%86%E7%9C%8B%E8%B3%87%E6%96%99%E7%9A%84%E5%BD%B1%E9%9F%BF-9aa2e2b420c6)
- [sklearn NMI](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html)


In [89]:
normalized_mutual_info_score(list(de_df_2.cpc_to_num), list(de_df_2.topic_to_num))

0.026732767037764147